In [2]:
import torch
import torch.nn.functional as F

ht = torch.tensor([[0.5,0.8,-0.2,0.1,0.9,-0.5,0.3,0.4]],dtype=float)

w_dq = torch.tensor([
    [1,0,1],
    [0,1,0],
    [-1,0,0],
    [0,-1,1],
    [1,1,0],
    [0,0,1],
    [0,1,0],
    [-1,0,-1],
],dtype=float)

w_dkv = torch.tensor([
    [1,0],
    [0,1],
    [1,0],
    [0,1],
    [0,0],
    [0,0],
    [1,1],
    [-1,-1],
],dtype=float)

ht.shape, w_dq.shape , w_dkv.shape

(torch.Size([1, 8]), torch.Size([8, 3]), torch.Size([8, 2]))

In [3]:
c_q = ht@w_dq
c_q

tensor([[ 1.2000,  1.9000, -0.3000]], dtype=torch.float64)

In [4]:
w_uq = torch.tensor([
    [1,0,0,1],
    [0,1,1,0],
    [0,0,1,1],
],dtype=float)
w_uq.shape

torch.Size([3, 4])

In [5]:
q_c = c_q@w_uq
q_c

tensor([[1.2000, 1.9000, 1.6000, 0.9000]], dtype=torch.float64)

In [6]:
w_qr = torch.tensor([[0.8,-0.3],
                     [0.2,0.9],
                     [-0.5,0.1]],dtype=float)
w_qr.shape

torch.Size([3, 2])

In [7]:
q_r_pre = c_q@w_qr
q_r_pos = torch.tensor([[-q_r_pre[0][1],q_r_pre[0][0]]])
q_r_pre,q_r_pos

(tensor([[1.4900, 1.3200]], dtype=torch.float64),
 tensor([[-1.3200,  1.4900]], dtype=torch.float64))

In [8]:
q = torch.concat([q_c,q_r_pos],dim=1)
q

tensor([[ 1.2000,  1.9000,  1.6000,  0.9000, -1.3200,  1.4900]],
       dtype=torch.float64)

In [9]:
c_kv = ht@w_dkv
c_kv

tensor([[0.2000, 0.8000]], dtype=torch.float64)

In [10]:
w_uk = torch.tensor([
    [1,0,1,0],
    [0,1,0,1]
],dtype=float)
w_uv = torch.tensor([
    [0,2,0,1],
    [1,0,2,0],
],dtype=float)

w_uk.shape , w_uk.shape


(torch.Size([2, 4]), torch.Size([2, 4]))

In [11]:
k_c = c_kv@w_uk
v = c_kv@w_uv
k_c,v

(tensor([[0.2000, 0.8000, 0.2000, 0.8000]], dtype=torch.float64),
 tensor([[0.8000, 0.4000, 1.6000, 0.2000]], dtype=torch.float64))

In [15]:
w_kr = torch.tensor([
    [0.6,-0.1],
    [0.1,0.7],
    [0.4,-0.2],
    [-0.3,0.8],
    [0.2,-0.5],
    [-0.6,0.3],
    [0.9,0.1],
    [-0.2,-0.7]
],dtype=float)

k_r_pre = ht@w_kr
k_r_pre

tensor([[ 0.9400, -0.2200]], dtype=torch.float64)

In [23]:
k_r_pos = torch.tensor([[-k_r_pre[0][1],k_r_pre[0][0]]])
k_r_pos

tensor([[0.2200, 0.9400]], dtype=torch.float64)

In [25]:
k  = torch.concat([k_c,k_r_pos],dim=1)
k

tensor([[0.2000, 0.8000, 0.2000, 0.8000, 0.2200, 0.9400]], dtype=torch.float64)

In [26]:
k_transpose= torch.transpose(k,dim0=0,dim1=1)
d_k = torch.tensor(k.shape[1])
d_k

tensor(6)

In [27]:
output = F.softmax((q@k_transpose)/torch.sqrt(d_k))@v
output # Same output as MLA

C:\Users\Parthiban\AppData\Local\Temp\ipykernel_7124\44781066.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax((q@k_transpose)/torch.sqrt(d_k))@v


tensor([[0.8000, 0.4000, 1.6000, 0.2000]], dtype=torch.float64)